In [1]:
import pandas as pd
from aiohttp import ClientSession
from fonctions.gestion_bdd import sauvegarde_bdd, lire_bdd_perso

In [4]:


session = ClientSession()

async with session.post('https://www.trackingthepros.com/d/list_players?filter_region=ALL&') as proplay:
    
    txt = await proplay.json(encoding='utf-8', content_type='text/html')


df_pro = pd.DataFrame(txt['data'])


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002CC531311E0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000002CC53116920>, 374017.656)]']
connector: <aiohttp.connector.TCPConnector object at 0x000002CC531337F0>


In [5]:
df_pro.drop(['DT_RowId', 'name', 'roleNum', 'accountsNum', 'online', 'gameID', 'onStream', 'team', 'onlineNum'], axis=1, inplace=True)

In [6]:
df_pro

,current,home,role,accounts,team_plug,plug,rankHigh,rankHighNum,rankHighLP,rankHighLPNum
0,North America,North America,Top,7,,Huni,Diamond I,24025.0,25,25
1,Korea,Korea,Top,5,,Untara,Master,25006.0,6,6
2,Japan,Korea,Jungle,8,Fukuoka-SoftBank-Hawks-gaming,Blank,Master,25242.0,242,242
3,Korea,Korea,Jungle,10,GenG,Peanut,Master,25488.0,488,488
4,Korea,Korea,Mid,4,T1,Faker,Grandmaster,26789.0,789,789
...,...,...,...,...,...,...,...,...,...,...
1999,China,China,Jungle,1,EDward-Gaming-Youth-Team,Coward,Grandmaster,26808.0,808,808
2000,Korea,Korea,ADC,1,KT-Rolster-Challengers,Hype,Challenger,28288.0,"1,288",1288
2001,Korea,Korea,Mid,1,Hanwha-Life-Esports-Academy,Tempester,Challenger,28113.0,"1,113",1113
2002,Liga Latinoamérica,Liga Latinoamérica,Jungle (Sub),1,Movistar-R7,Daiky,Master,25357.0,357,357


In [7]:
# NOTE : Prevoir 15m

df_final = pd.DataFrame(columns=['joueur', 'compte'])

for name_joueur in df_pro['plug']:

    try:
        df_rank = pd.read_html(f'https://www.trackingthepros.com/player/{name_joueur}/')[1][[0]]
        
        df_rank.columns = ['compte']
        
        df_rank['joueur'] = name_joueur 
        
        df_final = pd.concat([df_final, df_rank])
    except:
        print('erreur', f'{name_joueur}')
        continue

erreur Beyond
erreur Bless
erreur ZBB
erreur Conan
erreur Raison
erreur Greenie
erreur WaenA
erreur un1tback
erreur Blasting
erreur Rippii
erreur West
erreur Ruin
erreur Praelus
erreur Baltica
erreur MarioMe
erreur Zeicro
erreur Genthix
erreur Freire
erreur Atlanta
erreur Harbinger
erreur Beyond
erreur PiPiXuAn
erreur WXZ
erreur Joker
erreur Pink-Bean
erreur Empt2y
erreur wuji
erreur Jukes
erreur Call-Lin
erreur Yusui
erreur Fill
erreur Panda
erreur Seize
erreur CheonGo
erreur Juzo
erreur Mills
erreur Nap
erreur atyamomo
erreur Mental
erreur Aslan
erreur Wilder
erreur Looch
erreur Rogue
erreur Recap
erreur Ace
erreur Neo
erreur RayFarky
erreur Roki
erreur Broooock
erreur Yuhi
erreur Medic
erreur Kuma
erreur Olia
erreur Wing
erreur MGX
erreur Joker
erreur Pooh
erreur REFRA1N
erreur Kaiz
erreur Venus
erreur Piloto
erreur Maynah
erreur Júlia-Nakamura
erreur Cao
erreur bless
erreur Vichen
erreur NaMei
erreur fearness
erreur BadeMan
erreur Ley
erreur 997
erreur Windy
erreur poge
erreur Gang

In [8]:
# filtre

df_final = df_final[~df_final['compte'].str.contains('Inactive')]

df_final['region'] = df_final['compte'].str.extract(r'\[(.*?)\]', expand=False) # extrait les regions

df_final['compte'] = df_final['compte'].str.replace(r'\[.*?\]', '', regex=True) # les supprime des pseudos

df_final['compte'] = df_final['compte'].apply(lambda x : x[1:]) # laisse un espace vide

In [9]:
df_final

,joueur,compte,region
0,Huni,aM8hwcIFZCuQj0Lc,EUW
1,Huni,TSMFTX Heo,NA
2,Huni,뚱후니,KR
3,Huni,0Xr9AR1cV8sma05T,KR
4,Huni,Carb is my life,BR
...,...,...,...
0,Coward,Sad mood,KR
0,Hype,KT Hype,KR
0,Tempester,깔짝대기,KR
0,Daiky,3255263551006464,KR


In [10]:
sauvegarde_bdd(df_pro, 'data_proplayers')

In [11]:
sauvegarde_bdd(df_final, 'data_acc_proplayers')

In [2]:
df_data_pro = lire_bdd_perso('''SELECT data_acc_proplayers.*, data_proplayers.home, data_proplayers.role, data_proplayers.team_plug from data_acc_proplayers
                         LEFT JOIN data_proplayers ON data_acc_proplayers.joueur = data_proplayers.plug''', index_col=None).T

In [3]:
df_data_pro

,index,joueur,compte,region,home,role,team_plug
0,0,Huni,aM8hwcIFZCuQj0Lc,EUW,North America,Top,
1,1,Huni,TSMFTX Heo,NA,North America,Top,
2,2,Huni,뚱후니,KR,North America,Top,
3,3,Huni,0Xr9AR1cV8sma05T,KR,North America,Top,
4,4,Huni,Carb is my life,BR,North America,Top,
...,...,...,...,...,...,...,...
4241,0,Coward,Sad mood,KR,China,Jungle,EDward-Gaming-Youth-Team
4242,0,Hype,KT Hype,KR,Korea,ADC,KT-Rolster-Challengers
4243,0,Tempester,깔짝대기,KR,Korea,Mid,Hanwha-Life-Esports-Academy
4244,0,Daiky,3255263551006464,KR,Liga Latinoamérica,Jungle (Sub),Movistar-R7


In [9]:
thisPseudoListe = ['KC NEXT ADKING', 'Tomlora', 'Co24', 'TraYtoN']

In [15]:
observations_proplayers = ''
for joueur in thisPseudoListe:
    if joueur in df_data_pro['compte'].tolist():
        name_joueur = df_data_pro.loc[df_data_pro['compte'] == joueur, 'joueur'].values[0]
        role_joueur = df_data_pro.loc[df_data_pro['compte'] == joueur, 'role'].values[0]
        team_joueur = df_data_pro.loc[df_data_pro['compte'] == joueur, 'team_plug'].values[0]
        observations_proplayers += f':stadium: **{name_joueur}** : {role_joueur} chez {team_joueur} \n'